## ANN and residual gpr 

In [53]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import librosa
import pandas as pd

# 1. Feature extraction (same as before)
features = []
labels = []

for index, row in train_df.iterrows():
    file_name = row['filename']
    label = row['label']

    audio_file_path = f'Dataset/audios/train/{file_name}'
    audio, sr = librosa.load(audio_file_path, sr=None)

    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)

    spec_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
    spec_centroid_mean = np.mean(spec_centroid)

    zcr = librosa.feature.zero_crossing_rate(audio)
    zcr_mean = np.mean(zcr)

    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    feature_vector = np.hstack((mfcc_mean, spec_centroid_mean, zcr_mean, chroma_mean))
    features.append(feature_vector)
    labels.append(label)

features = np.array(features)
labels = np.array(labels)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

# 2. Train ANN
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(0.001), loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# 3. Predict with ANN
y_pred_ann_train = model.predict(X_train).flatten()
y_pred_ann_test = model.predict(X_test).flatten()

# 4. Compute residuals
residuals = y_train - y_pred_ann_train

# 5. Train GPR on residuals
kernel = C(1.0) * RBF(length_scale=1.0) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, normalize_y=True)
gpr.fit(X_train, residuals)

# 6. Predict residuals on test set
residuals_pred = gpr.predict(X_test)

# 7. Final prediction
y_final_pred = y_pred_ann_test + residuals_pred

# 8. Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_final_pred))
r2 = r2_score(y_test, y_final_pred)

print("Residual GPR + ANN Results:")
print(f"  RMSE: {rmse}")
print(f"  R-squared: {r2}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Residual GPR + ANN Results:
  RMSE: 0.9460778952754572
  R-squared: 0.33876169892740793


## SVM+residual_gpr

In [10]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import librosa
import pandas as pd


# 2. Train SVM
svm = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svm.fit(X_train, y_train)

# 3. Predict with SVM
y_pred_svm_train = svm.predict(X_train)
y_pred_svm_test = svm.predict(X_test)

# 4. Compute residuals
residuals = y_train - y_pred_svm_train

# 5. Train GPR on residuals
kernel = C(1.0) * RBF(length_scale=1.0) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, normalize_y=True)
gpr.fit(X_train, residuals)

# 6. Predict residuals on test set
residuals_pred = gpr.predict(X_test)

# 7. Final prediction
y_final_pred = y_pred_svm_test + residuals_pred

# 8. Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_final_pred))
r2 = r2_score(y_test, y_final_pred)

print("Residual GPR + SVM Results:")
print(f"  RMSE: {rmse}")
print(f"  R-squared: {r2}")


Residual GPR + SVM Results:
  RMSE: 0.8573752693807208
  R-squared: 0.45694212604792006


In [13]:
import joblib

# Save the trained SVM model
svm_model_filename = 'svm_model.pkl'
joblib.dump(svm, svm_model_filename)

# Save the trained GPR model
gpr_model_filename = 'gpr_model.pkl'
joblib.dump(gpr, gpr_model_filename)

# Save the scaler used for feature scaling
scaler_filename = 'scaler.pkl'
joblib.dump(scaler, scaler_filename)

print("Models and scaler saved successfully.")


Models and scaler saved successfully.


In [18]:
import pandas as pd
import librosa
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

# Load the test dataset
test_df = pd.read_csv('Dataset/test.csv')  # Update with the correct path

# Load the saved models
svm_model = joblib.load('svm_model.pkl')
gpr_model = joblib.load('gpr_model.pkl')

# Load the scaler used during training
scaler = joblib.load('scaler.pkl')

# Prepare lists to store features and filenames
test_features = []
test_filenames = []

# Extract features from the test audio files
for index, row in test_df.iterrows():
    file_name = row['filename']
    
    # Load the audio file
    audio_file_path = f'Dataset/audios/test/{file_name}'  # Update with correct path
    audio, sr = librosa.load(audio_file_path, sr=None)

    # Extract MFCC features (13 MFCC coefficients)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)  # Mean of each MFCC coefficient

    # Extract Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
    spectral_centroid_mean = np.mean(spectral_centroid)

    # Extract other features like zero-crossing rate, spectral roll-off, etc. (if applicable)
    # Example: Zero-crossing rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)
    zero_crossing_rate_mean = np.mean(zero_crossing_rate)

    # Example: Chroma features
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    # Combine features into a single vector (total features might add up to 27)
    feature_vector = np.hstack((mfcc_mean, spectral_centroid_mean, zero_crossing_rate_mean, chroma_mean))

    # Append the feature vector and filename
    test_features.append(feature_vector)
    test_filenames.append(file_name)

# Convert to numpy array
test_features = np.array(test_features)

# Check if the number of features is correct (27 features)
print(f"Test features shape: {test_features.shape}")

# Normalize the features using the same scaler as during training
test_features_scaled = scaler.transform(test_features)

# Make predictions using the SVM model
svm_predictions = svm_model.predict(test_features_scaled)

# Make residual predictions using the GPR model
residuals_pred = gpr_model.predict(test_features_scaled)

# Combine the SVM predictions and GPR residuals
final_predictions = svm_predictions + residuals_pred

# Convert predictions to DataFrame for submission
submission_df = pd.DataFrame({
    'filename': test_filenames,
    'predicted_label': final_predictions
})

# Check the first few predictions
print(submission_df.head())

# Save predictions in the required format for submission
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved in 'submission.csv'!")


Test features shape: (204, 27)
         filename  predicted_label
0   audio_804.wav         4.042393
1  audio_1028.wav         3.702302
2   audio_865.wav         3.277618
3   audio_774.wav         2.384411
4  audio_1138.wav         4.150069
Predictions saved in 'submission.csv'!


### here got 0.632 score

## Recursive implementation 
🔁 What You're Proposing
Let’s say you have your initial features X. You want to:

Train SVR on X → y

Predict y_pred_1

Concatenate y_pred_1 to X → X' = [X, y_pred_1]

Train GPR on X' → y

Predict y_pred_2, then concatenate y_pred_2 to get X'' = [X, y_pred_1, y_pred_2]

Repeat this for 5 steps — each time appending the previous prediction as a feature and retraining.

In [51]:
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Make a copy of the original data
X_train_stack = X_train.copy()
X_test_stack = X_test.copy()

current_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
current_model.fit(X_train_stack, y_train)
y_pred_train = current_model.predict(X_train_stack)
y_pred_test = current_model.predict(X_test_stack)

# Add prediction as a new feature
X_train_stack = np.hstack((X_train_stack, y_pred_train.reshape(-1, 1)))
X_test_stack = np.hstack((X_test_stack, y_pred_test.reshape(-1, 1)))

n_iters = 4  # Already did SVR once, do 4 more

for i in range(n_iters):
    kernel = C(1.0) * RBF(length_scale=1.0) + WhiteKernel()
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, normalize_y=True)
    gpr.fit(X_train_stack, y_train)
    
    y_pred_train = gpr.predict(X_train_stack)
    y_pred_test = gpr.predict(X_test_stack)
    
    # Add predictions to the stack
    X_train_stack = np.hstack((X_train_stack, y_pred_train.reshape(-1, 1)))
    X_test_stack = np.hstack((X_test_stack, y_pred_test.reshape(-1, 1)))

# Final evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2 = r2_score(y_test, y_pred_test)

print(f"Recursive Feedback Prediction (5 layers):")
print(f"  RMSE: {rmse}")
print(f"  R-squared: {r2}")


Recursive Feedback Prediction (5 layers):
  RMSE: 0.9134751219086583
  R-squared: 0.3835502816919447


## not good  Approach

✅ Full Pipeline: SVM + Residual GPR → ANN

In [59]:
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import librosa
import pandas as pd

X = np.array(features)
y = np.array(labels)

# 2. Scaling and splitting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3. SVM + Residual GPR
svm = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svm.fit(X_train, y_train)
y_pred_svm_train = svm.predict(X_train)
y_pred_svm_test = svm.predict(X_test)

residuals = y_train - y_pred_svm_train
kernel = C(1.0) * RBF(length_scale=1.0) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, normalize_y=True)
gpr.fit(X_train, residuals)

residuals_pred = gpr.predict(X_test)
svm_gpr_pred_test = y_pred_svm_test + residuals_pred

# 4. ANN using SVM+GPR output as input
X_ann_train = np.hstack([X_test, svm_gpr_pred_test.reshape(-1, 1)])
X_ann_test = X_ann_train  # Using same features for simplicity
y_ann_train = y_test  # true labels

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_ann_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])
model.compile(optimizer=Adam(0.001), loss='mse')
model.fit(X_ann_train, y_ann_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# 5. Predict with ANN
y_ann_pred = model.predict(X_ann_test).flatten()

# 6. Final evaluation
rmse = np.sqrt(mean_squared_error(y_test, y_ann_pred))
r2 = r2_score(y_test, y_ann_pred)

print("SVM + Residual GPR → ANN Results:")
print(f"  RMSE: {rmse}")
print(f"  R-squared: {r2}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
SVM + Residual GPR → ANN Results:
  RMSE: 0.4770611642155844
  R-squared: 0.8318672417250499


## the best model that i have got till now
SVM + Residual GPR → ANN Results:
* lets save it


In [60]:
# Save models
# Save SVM model
joblib.dump(svm, 'svm_model.pkl')
# Save GPR model
joblib.dump(gpr, 'gpr_model.pkl')
# Save ANN model
model.save('ann_model.h5')

print("Models saved successfully.")

Models saved successfully.


In [61]:
import pandas as pd
import librosa
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow.keras import losses

# Load the test dataset
test_df = pd.read_csv('Dataset/test.csv')  # Update with the correct path

# Load the saved models
svm_model = joblib.load('svm_model.pkl')
gpr_model = joblib.load('gpr_model.pkl')

# Load the scaler used during training
scaler = joblib.load('scaler.pkl')

# Load the pre-trained ANN model (ensure the loss function is passed explicitly)
ann_model = load_model('ann_model.h5', custom_objects={'mse': losses.MeanSquaredError()})

# Prepare lists to store features and filenames
test_features = []
test_filenames = []

# Extract features from the test audio files
for index, row in test_df.iterrows():
    file_name = row['filename']
    
    # Load the audio file
    audio_file_path = f'Dataset/audios/test/{file_name}'  # Update with the correct path
    audio, sr = librosa.load(audio_file_path, sr=None)

    # Extract MFCC features (13 MFCC coefficients)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)  # Mean of each MFCC coefficient

    # Extract Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
    spectral_centroid_mean = np.mean(spectral_centroid)

    # Extract other features like zero-crossing rate, spectral roll-off, etc. (if applicable)
    # Example: Zero-crossing rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)
    zero_crossing_rate_mean = np.mean(zero_crossing_rate)

    # Example: Chroma features
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    # Combine features into a single vector (total features might add up to 27)
    feature_vector = np.hstack((mfcc_mean, spectral_centroid_mean, zero_crossing_rate_mean, chroma_mean))

    # Append the feature vector and filename
    test_features.append(feature_vector)
    test_filenames.append(file_name)

# Convert to numpy array
test_features = np.array(test_features)

# Check if the number of features is correct (27 features)
print(f"Test features shape: {test_features.shape}")

# Normalize the features using the same scaler as during training
test_features_scaled = scaler.transform(test_features)

# Make predictions using the SVM model
svm_predictions = svm_model.predict(test_features_scaled)

# Make residual predictions using the GPR model
residuals_pred = gpr_model.predict(test_features_scaled)

# Combine the SVM predictions and GPR residuals
svm_gpr_pred_test = svm_predictions + residuals_pred

# Prepare the input data for the ANN model
X_ann_test = np.hstack([test_features_scaled, svm_gpr_pred_test.reshape(-1, 1)])

# Make predictions using the pre-trained ANN model
ann_predictions = ann_model.predict(X_ann_test).flatten()

# Convert predictions to DataFrame for submission
submission_df = pd.DataFrame({
    'filename': test_filenames,
    'predicted_label': ann_predictions
})

# Check the first few predictions
print(submission_df.head())

# Save predictions in the required format for submission
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved in 'submission.csv'!")


Test features shape: (204, 27)
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
         filename  predicted_label
0   audio_804.wav         1.882749
1  audio_1028.wav         3.698372
2   audio_865.wav         2.504019
3   audio_774.wav         1.914483
4  audio_1138.wav         2.941926
Predictions saved in 'submission.csv'!


## SVM->Ann+residual GPR

In [62]:
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as C
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

# Assuming you have features and labels loaded
X = np.array(features)  # Features (make sure this is a 2D array)
y = np.array(labels)    # Labels (make sure this is a 1D array)

# 1. Scaling and splitting data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 2. SVM + Residual GPR
svm = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svm.fit(X_train, y_train)

# Predict on the training and test data
y_pred_svm_train = svm.predict(X_train)
y_pred_svm_test = svm.predict(X_test)

# Calculate residuals (difference between true labels and SVM predictions)
residuals = y_train - y_pred_svm_train

# Create GPR model to model residuals
kernel = C(1.0) * RBF(length_scale=1.0) + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, normalize_y=True)
gpr.fit(X_train, residuals)

# Predict residuals on the test set using GPR
residuals_pred = gpr.predict(X_test)

# Combine SVM predictions and GPR residual predictions for final output
svm_gpr_pred_test = y_pred_svm_test + residuals_pred

# 3. ANN using SVM+GPR predictions as input
X_ann_train = np.hstack([X_test, svm_gpr_pred_test.reshape(-1, 1)])
X_ann_test = X_ann_train  # Using same features for simplicity
y_ann_train = y_test  # True labels for ANN training

# Define and compile the ANN model
ann_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_ann_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
])

ann_model.compile(optimizer=Adam(0.001), loss='mse')

# Train the ANN model
ann_model.fit(X_ann_train, y_ann_train, epochs=100, batch_size=16, validation_split=0.2, verbose=0)

# 4. Predict with ANN
y_ann_pred = ann_model.predict(X_ann_test).flatten()

# 5. Apply Residual GPR on ANN predictions
residuals_ann = y_ann_train - y_ann_pred
gpr_residual_ann = GaussianProcessRegressor(kernel=kernel, alpha=1e-4, normalize_y=True)
gpr_residual_ann.fit(X_ann_train, residuals_ann)

# Predict residuals on ANN predictions
residuals_pred_ann = gpr_residual_ann.predict(X_ann_test)

# Final prediction: ANN predictions + residuals from GPR
final_predictions = y_ann_pred + residuals_pred_ann

# 6. Final evaluation
rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
r2 = r2_score(y_test, final_predictions)

print("SVM + Residual GPR → ANN → Residual GPR Results:")
print(f"  RMSE: {rmse}")
print(f"  R-squared: {r2}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
SVM + Residual GPR → ANN → Residual GPR Results:
  RMSE: 0.294985932600357
  R-squared: 0.9357154183806489


### best model till now 
*SVM + Residual GPR → ANN → Residual GPR Results:

In [63]:
# Save the models
# Save the SVR model using joblib
joblib.dump(svm, 'svm_model.pkl')

# Save the GPR model using joblib
joblib.dump(gpr, 'gpr_model.pkl')

# Save the ANN model using Keras' .save() method
ann_model.save('ann_model.h5')

print("Models have been saved!")

Models have been saved!


In [66]:
import pandas as pd
import librosa
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from tensorflow.keras import losses

# Load the test dataset
test_df = pd.read_csv('Dataset/test.csv')  # Update with the correct path

# Load the saved models
svm_model = joblib.load('svm_model.pkl')
gpr_model = joblib.load('gpr_model.pkl')

# Load the scaler used during training
scaler = joblib.load('scaler.pkl')

# Load the pre-trained ANN model (ensure the loss function is passed explicitly)
ann_model = load_model('ann_model.h5', custom_objects={'mse': losses.MeanSquaredError()})

# Prepare lists to store features and filenames
test_features = []
test_filenames = test_df['filename'].tolist()  # Extract filenames from test.csv

# Extract features from the test audio files
for file_name in test_filenames:
    # Load the audio file
    audio_file_path = f'Dataset/audios/test/{file_name}'  # Correct path to audio files
    audio, sr = librosa.load(audio_file_path, sr=None)

    # Extract MFCC features (13 MFCC coefficients)
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    mfcc_mean = np.mean(mfcc, axis=1)  # Mean of each MFCC coefficient

    # Extract Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)
    spectral_centroid_mean = np.mean(spectral_centroid)

    # Extract other features like zero-crossing rate, spectral roll-off, etc.
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio)
    zero_crossing_rate_mean = np.mean(zero_crossing_rate)

    # Example: Chroma features
    chroma = librosa.feature.chroma_stft(y=audio, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)

    # Combine features into a single vector (total features might add up to 27)
    feature_vector = np.hstack((mfcc_mean, spectral_centroid_mean, zero_crossing_rate_mean, chroma_mean))

    # Append the feature vector
    test_features.append(feature_vector)

# Convert to numpy array
test_features = np.array(test_features)

# Normalize the features using the same scaler as during training
test_features_scaled = scaler.transform(test_features)

# Make predictions using the SVM model
svm_predictions = svm_model.predict(test_features_scaled)

# Make residual predictions using the GPR model
residuals_pred = gpr_model.predict(test_features_scaled)

# Combine the SVM predictions and GPR residuals
svm_gpr_pred_test = svm_predictions + residuals_pred

# Prepare the input data for the ANN model
X_ann_test = np.hstack([test_features_scaled, svm_gpr_pred_test.reshape(-1, 1)])

# Make predictions using the pre-trained ANN model
ann_predictions = ann_model.predict(X_ann_test).flatten()

# Convert predictions to DataFrame for submission
submission_df = pd.DataFrame({
    'filename': test_filenames,
    'predicted_label': ann_predictions
})

# Save predictions in the required format for submission
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved in 'submission.csv'!")


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
Predictions saved in 'submission.csv'!
